# Zyra Notebook: Drought Animation Walkthrough (draft)

This walkthrough recreates the `samples/swarm/drought_animation.yaml` flow:

- Sync the last year of weekly drought frames from NOAA FTP.
- Scan frames metadata to identify cadence/missing timestamps.
- Fill gaps with a basemap-backed placeholder set.
- Compose an MP4 animation and write it locally.
- Optional narration via `narrate swarm` with mock-safe provider selection.
- Export pipeline/CLI equivalents for reproducibility.

Prereqs: Pillow for `process pad-missing`, FFmpeg on `PATH` for `visualize compose-video`, and access to the packaged basemap `pkg:zyra.assets/images/earth_vegetation.jpg` (avoids external downloads).


In [1]:
# LLM provider (optional narration; mock-safe fallback)
import os

# Prefer Ollama; set base/model here if not already exported in the env
os.environ.setdefault("ZYRA_LLM_PROVIDER", "ollama")
os.environ.setdefault("OLLAMA_BASE_URL", "http://host.docker.internal:11434")
os.environ.setdefault("LLM_MODEL", "gemma3:12b")

PROVIDER = os.environ.get("ZYRA_LLM_PROVIDER", "ollama")
AVAILABLE = {"ollama", "openai", "google", "mock"}
if PROVIDER not in AVAILABLE:
    PROVIDER = "ollama"


def choose_provider() -> str:
    has_openai = bool(os.environ.get("OPENAI_API_KEY"))
    has_google = bool(os.environ.get("GOOGLE_API_KEY"))
    has_ollama = bool(os.environ.get("OLLAMA_BASE_URL"))
    if PROVIDER == "openai" and has_openai:
        return "openai"
    if PROVIDER == "google" and has_google:
        return "google"
    if PROVIDER == "ollama" and has_ollama:
        return "ollama"
    return "mock"


LLM_PROVIDER = choose_provider()
LLM_MODEL = os.environ.get("LLM_MODEL")
print("LLM model:", LLM_MODEL)
print("LLM provider:", LLM_PROVIDER)

LLM model: gemma3:12b
LLM provider: ollama


In [2]:
# Notebook session + workspace
from pathlib import Path

from zyra.notebook import create_session

# Default workspace: ZYRA_NOTEBOOK_DIR -> /kaggle/working -> cwd
os.environ["ZYRA_NOTEBOOK_DIR"] = "/app/data"
sess = create_session()
WORKSPACE = sess.workspace()
DROUGHT_DIR = WORKSPACE / "drought_notebook"
DROUGHT_DIR.mkdir(parents=True, exist_ok=True)
print("Workspace root:", WORKSPACE)
print("Drought run dir:", DROUGHT_DIR)

Workspace root: /app/data
Drought run dir: /app/data/drought_notebook


In [3]:
# Paths and cadence defaults
FRAMES_RAW = DROUGHT_DIR / "frames_raw"
FRAMES_PADDED = DROUGHT_DIR / "frames_padded"
FRAMES_META = DROUGHT_DIR / "frames_meta.json"
VIDEO_OUT = DROUGHT_DIR / "drought_animation.mp4"
BASEMAP_REF = "pkg:zyra.assets/images/earth_vegetation.jpg"

FTP_PATH = "ftp://ftp.nnvl.noaa.gov/SOS/DroughtRisk_Weekly"
PATTERN = r"^DroughtRisk_Weekly_[0-9]{8}\.png$"  # single-escaped dot
CADENCE_SECONDS = 7 * 24 * 3600

for folder in (FRAMES_RAW, FRAMES_PADDED):
    folder.mkdir(parents=True, exist_ok=True)

print("Raw frames dir:", FRAMES_RAW)
print("Padded frames dir:", FRAMES_PADDED)

Raw frames dir: /app/data/drought_notebook/frames_raw
Padded frames dir: /app/data/drought_notebook/frames_padded


In [4]:
# Acquire drought frames (FTP sync for the past year)
import contextlib
import re
from datetime import datetime

from zyra.connectors.backends import ftp as ftp_backend
from zyra.utils.date_manager import DateManager


def frame_count(path: Path) -> int:
    return sum(1 for f in path.iterdir() if f.is_file())


date_start, _ = DateManager().get_date_range_iso("P1Y")
remote_filtered = (
    ftp_backend.list_files(
        FTP_PATH,
        pattern=PATTERN,
        since=date_start.isoformat(),
        date_format="%Y%m%d",
    )
    or []
)
dates = []
for name in remote_filtered:
    m = re.search(r"(\d{8})", name)
    if m:
        with contextlib.suppress(Exception):
            dates.append(datetime.strptime(m.group(1), "%Y%m%d"))
print(f"Remote frames past year: {len(remote_filtered)}")
if dates:
    print("Remote date span:", min(dates).date(), "to", max(dates).date())
if remote_filtered:
    print("Newest remote samples:", remote_filtered[-3:])

try:
    sess.acquire.ftp(
        path=FTP_PATH,
        sync_dir=str(FRAMES_RAW),
        pattern=PATTERN,
        since_period="P1Y",
        date_format="%Y%m%d",
    )
    print("Synced frames from FTP ->", FRAMES_RAW)
except Exception as exc:
    raise RuntimeError(f"FTP sync failed: {exc}") from exc

ready = frame_count(FRAMES_RAW)
print(f"Local frames downloaded: {ready}")
if ready:
    sample_local = sorted(FRAMES_RAW.iterdir())[-3:]
    print("Newest local files:", [p.name for p in sample_local])

Remote frames past year: 51
Remote date span: 2024-12-05 to 2025-11-20
Newest remote samples: ['DroughtRisk_Weekly_20251106.png', 'DroughtRisk_Weekly_20251113.png', 'DroughtRisk_Weekly_20251120.png']
Synced frames from FTP -> /app/data/drought_notebook/frames_raw
Local frames downloaded: 51
Newest local files: ['DroughtRisk_Weekly_20251106.png', 'DroughtRisk_Weekly_20251113.png', 'DroughtRisk_Weekly_20251120.png']


## Create a gap in the frames
We'll delete a couple of frames to see how pad-missing backfills missing timestamps.

In [5]:
# Remove two frames to simulate missing data
import random

frames = sorted(p for p in FRAMES_RAW.iterdir() if p.is_file())
if len(frames) < 2:
    print("Not enough frames to delete; skipping gap simulation")
else:
    to_delete = random.sample(frames, 2)
    for fp in to_delete:
        fp.unlink(missing_ok=True)
    print("Deleted frames:", [fp.name for fp in sorted(to_delete)])
    print("Remaining frame count:", sum(1 for f in FRAMES_RAW.iterdir() if f.is_file()))

Deleted frames: ['DroughtRisk_Weekly_20250109.png', 'DroughtRisk_Weekly_20250522.png']
Remaining frame count: 49


In [6]:
# Scan frames metadata (cadence, missing timestamps)
import json

meta_result = sess.transform.scan_frames(
    frames_dir=str(FRAMES_RAW),
    pattern=PATTERN,
    datetime_format="%Y%m%d",
    period_seconds=CADENCE_SECONDS,
    output=str(FRAMES_META),
)
summary = json.loads(FRAMES_META.read_text()) if FRAMES_META.exists() else {}
print("Frames meta:", FRAMES_META)
print(
    "Actual frames:",
    summary.get("frame_count_actual"),
    "Missing:",
    summary.get("missing_count"),
)

Frames meta: /app/data/drought_notebook/frames_meta.json
Actual frames: 49 Missing: 2


In [7]:
# Fill missing frames using basemap placeholders
import shutil

# Start padded dir fresh and seed it with existing frames
if FRAMES_PADDED.exists():
    shutil.rmtree(FRAMES_PADDED)
FRAMES_PADDED.mkdir(parents=True, exist_ok=True)
for src in FRAMES_RAW.iterdir():
    if src.is_file():
        shutil.copy2(src, FRAMES_PADDED / src.name)

pad_result = sess.process.pad_missing(
    frames_meta=str(FRAMES_META),
    output_dir=str(FRAMES_PADDED),
    fill_mode="basemap",
    basemap=BASEMAP_REF,
    overwrite=True,
)
filled = sum(1 for f in FRAMES_PADDED.iterdir() if f.is_file())
print("Padded frames dir:", FRAMES_PADDED)
print("Total frames after padding:", filled)

Padded frames dir: /app/data/drought_notebook/frames_padded
Total frames after padding: 51


In [8]:
# Compose MP4 animation from padded frames (requires ffmpeg on PATH)
video_path = sess.visualize.compose_video(
    frames=str(FRAMES_PADDED),
    output=str(VIDEO_OUT),
    fps=4,
    basemap=BASEMAP_REF,
)
print("Video path:", video_path)

Video path: /app/data/drought_notebook/drought_animation.mp4


In [9]:
# Save final animation locally (mirrors disseminate/local stage)
final_path = sess.disseminate.local(
    input=str(VIDEO_OUT),
    path=str(VIDEO_OUT),
)
print("Local copy:", final_path)

Local copy: /app/data/drought_notebook/drought_animation.mp4


In [10]:
# Optional narration of the drought animation
narration_meta = summary if isinstance(summary, dict) else {}
missing_dates = [
    str(ts).split("T")[0] for ts in narration_meta.get("missing_timestamps") or []
]
missing_dates_str = ", ".join(missing_dates) if missing_dates else "none"
narration_input = {
    "title": "Weekly drought risk animation",
    "description": f"Summarize the NOAA drought animation, note where frames were padded, and list the padded weeks: {missing_dates_str}.",
    "data": {
        "frame_count_expected": narration_meta.get("frame_count_expected"),
        "frame_count_actual": narration_meta.get("frame_count_actual"),
        "missing_count": narration_meta.get("missing_count"),
        "missing_timestamps": narration_meta.get("missing_timestamps") or [],
        "frame_count_after_padding": filled,
        "padded_weeks": missing_dates,
    },
}
try:
    narration = sess.narrate.swarm(
        provider=LLM_PROVIDER,
        model=LLM_MODEL,
        base_url=os.environ.get("OLLAMA_BASE_URL"),
        preset="scientific_lite",
        input_data=narration_input,
    )
    print("Narration:", narration)
except Exception as exc:
    print("Narrate/swarm not executed:", exc)

Narration: The NOAA drought animation includes padded frames for weeks January 9, 2025, and May 22, 2025.


In [11]:
# Export pipeline + CLI equivalents for reproducibility
pipeline = sess.to_pipeline()
cli_cmds = sess.to_cli()

pipeline_path = DROUGHT_DIR / "drought_pipeline.json"
pipeline_path.write_text(json.dumps(pipeline, indent=2))
print("Pipeline saved to", pipeline_path)
print("CLI commands:")
for cmd in cli_cmds:
    print(cmd)

Pipeline saved to /app/data/drought_notebook/drought_pipeline.json
CLI commands:
zyra acquire ftp --path ftp://ftp.nnvl.noaa.gov/SOS/DroughtRisk_Weekly --sync-dir /app/data/drought_notebook/frames_raw --pattern ^DroughtRisk_Weekly_[0-9]{8}\.png$ --since-period P1Y --date-format %Y%m%d --workdir /app/data --output /app/data/acquire_ftp.tmp
zyra transform scan-frames --frames-dir /app/data/drought_notebook/frames_raw --pattern ^DroughtRisk_Weekly_[0-9]{8}\.png$ --datetime-format %Y%m%d --period-seconds 604800 --output /app/data/drought_notebook/frames_meta.json --workdir /app/data
zyra process pad-missing --frames-meta /app/data/drought_notebook/frames_meta.json --output-dir /app/data/drought_notebook/frames_padded --fill-mode basemap --basemap pkg:zyra.assets/images/earth_vegetation.jpg --overwrite --workdir /app/data
zyra visualize compose-video --frames /app/data/drought_notebook/frames_padded --output /app/data/drought_notebook/drought_animation.mp4 --fps 4 --basemap pkg:zyra.assets/